In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

config_path = '/opt/ml/baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'  # config 경로 설정

# config file 들고오기
cfg = Config.fromfile(config_path)

# 경로 설정

root='/opt/ml/dataset/'

checkpoint_name = 'best_bbox_mAP_50_epoch_'  # best model weights name 설정

test_file = 'test.json'

work_dir = '/opt/ml/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash'  # workdir 경로 설정

# dataset config 수정

test_resize = (512,512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + test_file
cfg.data.test.pipeline[1]['img_scale'] = test_resize # Resize
cfg.data.test.test_mode = True  # test mode 전환 (annotation을 불러오지 않는다.)

# Runtime 설정

batch_size = 32
seed = 18

cfg.data.samples_per_gpu = batch_size
cfg.seed = seed

cfg.gpu_ids = [1]

cfg.work_dir = work_dir

# model 설정
cfg.model.roi_head.bbox_head.num_classes = len(classes)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)  # ?
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{checkpoint_name}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES  # 모델에서 예측해야할 클래스 정의
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash/best_bbox_mAP_50_epoch_3.pth


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 28.0 task/s, elapsed: 174s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{checkpoint_name}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.11330428 29.530052 122.55136 261.12283 332...,test/0000.jpg
1,0 0.40133885 713.76 637.89624 908.5371 986.304...,test/0001.jpg
2,0 0.12081697 667.0104 229.33292 850.3711 343.8...,test/0002.jpg
3,0 0.08328196 34.739044 307.72137 956.47766 772...,test/0003.jpg
4,0 0.23241846 308.77057 255.81955 875.65765 762...,test/0004.jpg
